In [1]:
import tensorflow as tf 
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd 
import numpy as np


In [2]:
#  wee need to load a trained model and  scaler file
model = load_model("model.h5")  

#  we will load  our pickle file in read bit mode 
with open("ohe_geo.pkl",'rb') as file:
    lable_enc_geo=pickle.load(file)

with open('lable_enc_gen.pkl','rb') as file:
    lable_enc_gender= pickle.load(file)

with open('scaler.pkl','rb')as file:
    scaler=pickle.load(file)

In [3]:

lable_enc_gender.classes_

array(['Female', 'Male'], dtype=object)

In [4]:
inputdata = {
     "CreditScore" : 618,
     "Geography"	: "France",
     "Gender"	: "Male",
     "Age"	: 24,
     "Tenure": 2,
     "Balance":	200000,
     "NumOfProducts":1,	
     "HasCrCard":1,	
     "IsActiveMember"	:1,
     "EstimatedSalary"	:1000 
     
     
     
}

In [5]:
geo_encoded =  lable_enc_geo.transform([[inputdata["Geography"]]]).toarray()
geo_encoded_df= pd.DataFrame(geo_encoded,columns=lable_enc_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

c:\Users\ACER\virtual_venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
import pandas as pd
input_df = pd.DataFrame([inputdata])

In [7]:
#  will will encode gender 
input_df["Gender"] = lable_enc_gender.transform(input_df["Gender"])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,618,France,1,24,2,200000,1,1,1,1000


In [8]:
# merged_df = pd.concat([input_df.reset_index(drop=True),
#                        geo_encoded_df.reset_index(drop=True)], axis=1)

In [9]:
# merged_df


In [10]:
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,618,1,24,2,200000,1,1,1,1000,1.0,0.0,0.0


In [11]:
# sceling input data 
input_sceler = scaler.transform(input_df)
input_sceler

array([[-0.349186  ,  0.91324755, -1.41636539, -1.04241787,  1.98372734,
        -0.91668767,  0.64920267,  0.97481699, -1.72877905,  1.00150113,
        -0.57946723, -0.57638802]])

In [12]:
#  now we will predict wether a person is likely to churn 
prediction = model.predict(input_sceler)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step


array([[0.01954402]], dtype=float32)

In [13]:
# prediction prebablity 
prediction_prob = prediction[0][0]
prediction_prob

np.float32(0.019544024)

In [14]:
if prediction_prob > 0.5 :
    print("the person is likely to churn")

else :
    print("the person will not churn ")    


the person will not churn 
